# 앙상블 모형 실습

#### reference
[참고1](https://velog.io/@changhtun1/ensemble#%EA%B8%B0%EC%B4%88%EB%B6%80%ED%84%B0-%EC%8C%93%EC%95%84%EA%B0%80%EB%8A%94-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-7)  
[참고2](https://blog.naver.com/winddori2002/221829427442)  
[참고3](https://teddylee777.github.io/machine-learning/ensemble%EA%B8%B0%EB%B2%95%EC%97%90-%EB%8C%80%ED%95%9C-%EC%9D%B4%ED%95%B4%EC%99%80-%EC%A2%85%EB%A5%98-1)  
[참고4](https://hyemin-kim.github.io/2020/08/04/S-Python-sklearn4/)

**앙상블이란?**
- 여러 개의 모델들을 활용하여 더 강력한 성능의 모델을 만드는 기법

**앙상블 기법의 종류**
- 보팅 (Voting): 투표를 통해 결과 도출
- 배깅 (Bagging): 샘플 중복 생성을 통해 결과 도출
- 부스팅 (Boosting): 이전 오차를 보완하면서 가중치 부여
- 스태킹 (Stacking): 여러 모델을 기반으로 예측된 결과를 통해 meta 모델이 다시 한번 예측

---
## Voting
[voting 참고](https://yganalyst.github.io/ml/ML_chap6-1/#1-1-%EC%A7%81%EC%A0%91-%ED%88%AC%ED%91%9C)  

- 투표를 통해 결정하는 방식
- 서로 다른 알고리즘이 도출해 낸 결과물에 대하여 최종 투표하는 방식
- hard/soft vote로 나뉨
    - hard: 결과물에 대한 최종 값을 투표를 통해 결정(다수결)
    - soft: 각 레이블의 예측 확률의 평균으로 최종 분류 결정  
    cf. 보통 대회에서는 soft vote 방식이 더 합리적이다.
    
### >> 분류(classification)

In [1]:
# 필요한 패키지 로드
from sklearn.datasets import load_iris
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
# 데이터셋 로드
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3,
                                                   random_state=2021,
                                                   shuffle=True)

**hard voting**

In [3]:
# 약한 학습기 구축
log_model = LogisticRegression(max_iter= 1000) # Gradient Descent 방식 반복 횟수 1000회
rnd_model = RandomForestClassifier()
svm_model = SVC()

# 앙상블 모델 구축(hard voting)
voting_model = VotingClassifier(
    estimators=[('lr',log_model),('rf',rnd_model),('svc',svm_model)], # 3개의 약한 학습기
    voting='hard' # 직접 투표(hard voting)
)

# 앙상블 모델 학습
voting_model.fit(X_train,y_train)

# 모델 비교
for model in (log_model, rnd_model, svm_model, voting_model):
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    print(model.__class__.__name__, ": {}".format(accuracy_score(y_test, predict)))

LogisticRegression : 1.0
RandomForestClassifier : 0.9333333333333333
SVC : 0.9777777777777777
VotingClassifier : 1.0


**soft voting**

In [4]:
# 약한 학습기 구축
log_model = LogisticRegression(max_iter= 1000) # Gradient Descent 방식 반복 횟수 1000회
rnd_model = RandomForestClassifier()
svm_model = SVC(probability=True) # soft 방식을 위해서 probability 옵션 지정 필요

# 앙상블 모델 구축(soft voting)
voting_model = VotingClassifier(
    estimators=[('lr',log_model),('rf',rnd_model),('svc',svm_model)], # 3개의 약한 학습기
    voting='soft' # 간접 투표(soft voting)
)

# 앙상블 모델 학습
voting_model.fit(X_train,y_train)

# 모델 비교
for model in (log_model, rnd_model, svm_model, voting_model):
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    print(model.__class__.__name__, ": {}".format(accuracy_score(y_test, predict)))

LogisticRegression : 1.0
RandomForestClassifier : 0.9333333333333333
SVC : 0.9777777777777777
VotingClassifier : 0.9777777777777777


### >> 회귀 (Regression)

*voting regression*은 **voting 방식(hard or soft)을 설정하지 않는다.**

아래는 보스턴 데이터를 이용한 예시이다.

In [5]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.datasets import load_boston

# 데이터셋 로드
boston = load_boston()
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3,
                                                   random_state=1, shuffle=True)

# 약한 학습기 구축
lng_model = LinearRegression()
ridge_model = Ridge(alpha=0.1)
lasso_model = Lasso()
elt_model = ElasticNet()

# 앙상블 모델 구축(hard voting)
voting_model = VotingRegressor(
    estimators= [('linear', lng_model), ('svr', ridge_model), 
                 ('rfr', lasso_model), ('elt', elt_model)],
    weights=[1,2,1,1])

# 앙상블 모델 학습
# voting_model.fit(X_train, y_train)

# 모델 학습 및 평가
from sklearn.metrics import mean_squared_error, r2_score

for model in (lng_model, ridge_model, lasso_model, elt_model, voting_model):
    model.fit(X_train, y_train)
    predcit = model.predict(X_test)
    print(model.__class__.__name__, ": {}".format(mean_squared_error(y_test, predcit)))

LinearRegression : 19.831323672063235
Ridge : 19.69619983181413
Lasso : 30.29379822196717
ElasticNet : 27.513171154748665
VotingRegressor : 22.319758694392764


---
아래는 *make_moons* 함수를 이용해서 가상데이터를 만들어 *voting* 방식을 실습한 예시이다.

In [6]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 약한 학습기 구축
log_clf = LogisticRegression(solver = 'liblinear') # 최적화에 사용할 알고리즘 설정
rnd_clf = RandomForestClassifier(n_estimators=10) # 생성할 tree 개수
svm_clf = SVC(gamma='auto')

# 앙상블 모델 구축
voting_clf = VotingClassifier(estimators=[('lr',log_clf),
                                         ('rf',rnd_clf),
                                         ('svc',svm_clf)],
                             voting='hard')

# 앙상블 모델 학습
# voting_clf.fit(X_train, y_train)

# 모델 학습 및 평가
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, ":", accuracy_score(y_test,y_pred))

LogisticRegression : 0.864
RandomForestClassifier : 0.88
SVC : 0.888
VotingClassifier : 0.888


---

## Bagging(Bootstrap Aggregating)

- 하나의 모델을 다양하게 학습
- Bootstrap은 여러 개의 dataset을 중첩 허용하게 하여 샘플링하여 분할하는 방식이며 Aggregating은 집계를 의미
> ex. 데이터 셋의 구성이 [1, 2, 3, 4, 5 ]로 되어 있다면,  
> - group 1 = [1, 2, 3]  
> - group 2 = [1, 3, 4]  
> - group 3 = [2, 3, 5]  
> 로 구성된다고 말할 수 있다.
- 동일 데이터로부터 복원 랜덤 샘플링을 통해 여러 개의 데이터 셋을 만들고 하나의 모델로 각 데이터에 학습(중복을 허용하므로 같은 훈련 샘플이 여러 예측기에 사용될 수 있음)
- 분류의 경우 다수결이고, 회귀의 경우는 각 모델 결과의 평균값을 최종값으로 출력

**대표 기법**
- Bagging
- Random Forest(Decision Tree 기반 Bagging 앙상블)

## Pasting
- 중복을 허용하지 않고 샘플링하는 방식
- bootstrap 옵션을 이용해서 설정(True이면 배깅 방식, False이면 페이스팅 방식)

아래는 *make_moons* 함수를 이용해서 가상데이터를 만들어 *bagging* 방식을 실습한 예시이다.

In [7]:
# moon dataset
from sklearn.datasets import make_moons
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=500, noise = .3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 모델 생성
dt_model = DecisionTreeClassifier(random_state=42)
bag_model = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500, # 앙상블에 사용할 분류기 개수
    max_samples=100, # 하나의 예측기에 들어가는 샘플수 설정
    bootstrap=True, # True(중복허용, 배깅, default), False(중복허용 X, 페이스팅)
    n_jobs=-1
)

# 모델 학습 및 평가
for model in (dt_model, bag_model):
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    print(model.__class__.__name__, ':', accuracy_score(y_test, predict))

DecisionTreeClassifier : 0.856
BaggingClassifier : 0.912


### + oob 평가

배깅을 사용하면 어떤 샘플은 여러번 샘플링되고, 어떤 것은 전혀 사용되지 않을 수도 있다.

*BaggingClassifier*는 중복을 허용하여 훈련 세트의 크기 만큼 m개의 샘플을 선택하는데, 이는 평균적으로 각 예측기에 훈렴샘플의 **63%** 정도만 샘플링 된다는 것을 의미한다.  
여기서 선택되지 않은 훈련 샘플의 나머지 **37%** 정도를 **oob(out-of-bag)샘플**이라고 부른다.

핵심은 예측기가 훈련되는 동안 이 **oob샘플**을 사용하지 않으므로 검증 세트나 교차검증을 사용하지 않고 이를 이용해 평가할 수 있다.

앙상블의 평가는 각 예측기의 **oob평가**를 평균하여 얻는다.

사이킷런에서 *BaggingClassifier*의 *oob_score=Tru*e로 지정하면 훈련을 마치고 **oob평가**를 수행한다.

In [116]:
bag_model = BaggingClassifier(DecisionTreeClassifier(),
                             n_estimators=500,
                             bootstrap=True,
                             n_jobs=-1,
                             oob_score=True) # oob 평가 수행
bag_model.fit(X_train, y_train)
bag_model.oob_score_   # 500개 결정트리 분류기의 oob점수를 평균한 값

0.8906666666666667

약 90%의 정확도를 보여준다.  
이는 테스트 데이터셋을 사용하지 않고 훈련데이터에서 사용되지 않은 것을 재활용(?)한 것으로, 실제 테스트 데이터셋과 비교해보자.

In [117]:
predict = bag_model.predict(X_test)
accuracy_score(y_test, predict)

0.904

실제로 비슷한 정확도를 보여준다.

**oob평가**를 통해 얻은 결정 함수의 값(범주에 속할 확률)은 *oob_decision_function_* 에서 확인할 수 있다(예측기가 *predict_proba()* 메서드를 가지므로)

In [119]:
# 총 375(훈련 데이터셋 크기)개 중 10개만 
bag_model.oob_decision_function_[:10]

array([[0.36649215, 0.63350785],
       [0.35393258, 0.64606742],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.07526882, 0.92473118],
       [0.40883978, 0.59116022],
       [0.00555556, 0.99444444],
       [0.98492462, 0.01507538],
       [0.98360656, 0.01639344]])

## Random Forest

[랜덤포레스트 참고1](https://yganalyst.github.io/ml/ML_chap6-3/)  
[랜덤포레스트 참고2](https://jhryu1208.github.io/data/2020/11/16/ML_decision_tree_ensemble_random_forest/)  
[랜덤포테스트 참고3](https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-5-%EB%9E%9C%EB%8D%A4-%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8Random-Forest%EC%99%80-%EC%95%99%EC%83%81%EB%B8%94Ensemble)
- Bagging 기법에 Decission Tree 적용한 것과 같음(배깅방법을 사용한 결정트리 앙상블 모델)
- 의사결정나무의 모임(n_estimators 옵션으로 개수 설정)
- 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 것이 아니라, 무작위로 선택한 특성들 중에서 최선의 특성을 찾는 방식을 채택하여 무작위성을 더 갖음

중요 매개변수는 *n_estimators*, *max_features*이고, *max_depth* 같은 사전 가지치기 옵션이 있다.

- *n_estimators*는 클수록 좋다.
> 왜냐하면 더 많은 트리를 평균하면 과대적합을 줄여 더 안정적인 모델을 만들기 때문이다.
하지만 이로 인해 잃는 것도 있는데, 더 많은 트리는 더 많은 메모리와 긴 훈련 시간으로 이어진다.

- *max_features*는 각 트리가 얼마나 무작위가 될지를 결정하며,작은 *max_features*는 과대적합을 줄여준다. 하지만, 일반적으로 아래의 기본값을 쓰는 것이 좋은 방법이다.
> 분류 : *max_features=sqrt(n_features)*  
> 회귀 : *max_features=n_features*

*max_features*나 *max_leaf_nodes* 매개변수를 추가하면 가끔 성능이 향상되기도 한다.


### 1) ExtraTree
- 랜덤포레스트는 각 노드에서 무작위로 특성을 뽑은 다음 최적의 특성과 임계값을 선택한다.
- 하지만 엑스트라 트리는 최적의 특성과 임계값을 찾는것 대신, 후보 특성을 사용해 무작위로 분할한 다음에 최상의 분할 선택
- 이렇게되면 기본적으로 편향이 많은 랜던포레스트보다 더욱 편향이 심해지지만, 분산을 더욱 낮출 수 있음
- 트리 알고리즘에서는 모든 노드에서 최적의 특성과 임계값을 고르는데 시간이 많이 들지만, 엑스트라 트리를 사용하면 훈련과 예측속도가 빠름
- 엑스트라 트리는 *ExtraTreesClassifier*를 이용
- *RandomForestClassifier*와 *ExtraTreesClassifier* 중 어떤 것이 더 좋을지는 판단하기 어렵기 때문에, 교차검증을 통해서 서로 비교해보고, 더 나은 모델을 선택하여 그리드 탐색방법을 사용해 하이퍼파라미터 튜닝 수행

### 2) 특성 중요도
- 랜덤포레스트는 성능이 좋다는 장점말고, 특성의 상대적 중요도를 측정하기 쉬움(트리기반 모델은 특성 중요도 제공)
- 사이킷런에서는 어떤 특성을 사용한 노드가 평균적으로 불순도를 감소시키는지 확인하여 특성 중요도를 측정하고, 훈련이 끝나고 난 뒤에 특성마다 자동으로 점수를 계산하고 저장
- 저장된 값은 *featureimportances* 변수에 저장되어 있음

In [136]:
# 필요한 패키지 로드
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier

# 데이터셋 로드
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3,
                                                   random_state=42,
                                                   stratify=y)
# 모델 생성 및 학습
model = RandomForestClassifier(
    n_estimators=500,
    n_jobs=-1)
model.fit(X_train, y_train)

# 평가
predict = model.predict(X_test)
print('학습 데이터 평가: {}'.format(model.score(X_train, y_train)))
print('평가 데이터 평가: {}'.format(model.score(X_test, y_test)))

학습 데이터 평가: 1.0
평가 데이터 평가: 0.9473684210526315


다음 코드는 iris데이터셋에 랜덤포레스트를 훈련시키고 중요도를 출력한 것이다.

In [138]:
from sklearn.datasets import load_iris
iris = load_iris()
model = RandomForestClassifier(n_estimators=500, n_jobs=-1)
model.fit(iris.data, iris.target)
for name, score in zip(iris.feature_names, model.feature_importances_):
    print(name, score)

sepal length (cm) 0.09471873135932563
sepal width (cm) 0.024490913795713644
petal length (cm) 0.4408647568319765
petal width (cm) 0.4399255980129843


변수는 총 4개이고, 위의 4개 점수를 합하면 1이된다.  
즉, 꽃잎의 길이(petal length)와 꽃잎의 너비(petal width)가 각각 45%, 43%로 가장 높다.